<a href="https://colab.research.google.com/github/saravanakumarjsk/Simple-torch-scripts-/blob/master/complete_torch_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

    

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

   

!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json

    
!pip install -q kaggle
!pip install -q kaggle-cli

Saving kaggle.json to kaggle.json
kaggle.json
total 4
-rw------- 1 root root 72 Nov 18 16:22 kaggle.json
     |████████████████████████████████| 81kB 2.6MB/s 
     |████████████████████████████████| 5.3MB 6.5MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 112kB 62.8MB/s 
     |████████████████████████████████| 112kB 76.2MB/s 


In [2]:
!kaggle datasets download -d puneet6060/intel-image-classification

 97% 337M/346M [00:08<00:00, 47.0MB/s]
100% 346M/346M [00:08<00:00, 43.7MB/s]


In [5]:
%ls

intel-image-classification.zip  sample_data/  seg_test/
kaggle.json                     seg_pred/     seg_train/


In [0]:
from zipfile import ZipFile as z 

with z('intel-image-classification.zip', 'r') as w:
	w.extractall()

In [13]:
%cd '/content'

/content


In [10]:
%cd '/content/seg_test/seg_test/buildings/'

/content/seg_test/seg_test/buildings


In [0]:
# import useful libraries
import numpy as np # linear algebra
from matplotlib import pyplot as plt
from PIL import Image
from glob import glob
import os

# import pytorch modules
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.datasets import ImageFolder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
train_path = '/content/seg_train/seg_train/'
test_path = '/content/seg_test/seg_test/'
valid_path = '/content/seg_pred/seg_pred/'

In [0]:
transformer = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5])
])

train_loader = DataLoader(
    ImageFolder(train_path, transform=transformer),
    num_workers=8, batch_size=200, shuffle=True
)

test_loader = DataLoader(
    ImageFolder(test_path, transform=transformer),
    num_workers=8, batch_size=200, shuffle=True
)

In [6]:
resnet_model = models.resnet50(pretrained=True)

# freeze all parameters in ResNet so we won't have to retrain them
for param in resnet_model.parameters():
    param.requires_grad = False

# replace the last layer with another one that will be trainable
in_shape = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(in_shape, 6)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 190MB/s]


In [7]:
error = nn.CrossEntropyLoss()
opt = optim.Adam(resnet_model.parameters())

def train(model, train_loader, n_epochs=100):
    model = model.to(device)
    model.train()
    
    for epoch in range(n_epochs):
        epoch_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            out = model(x)
            loss = error(out, y)
            loss.backward()
            opt.step()
            epoch_loss += loss.item()        
        if epoch % int(0.1*n_epochs) == 0:
            print(f'epoch: {epoch} \t Train Loss: {epoch_loss:.4g}')
            
train(resnet_model, train_loader)

epoch: 0 	 Train Loss: 41.43
epoch: 10 	 Train Loss: 16.07
epoch: 20 	 Train Loss: 14.22
epoch: 30 	 Train Loss: 12.44
epoch: 40 	 Train Loss: 11.46
epoch: 50 	 Train Loss: 10.68
epoch: 60 	 Train Loss: 9.983
epoch: 70 	 Train Loss: 9.592
epoch: 80 	 Train Loss: 9.014
epoch: 90 	 Train Loss: 8.837


In [0]:
torch.save(resnet_model.state_dict(), 'model.pth')

In [8]:
def test(model, test_loader):
    model = model.to(device)
    model.eval()
    loss = 0
    for x, y in test_loader:
        with torch.no_grad():
            out = model(x.to(device)).cpu()
        loss += error(out, y)
        
    print(f'Test loss: {loss:.4g}')
    
test(resnet_model, test_loader)

Test loss: 4.304
